In [2]:
north_pole = (90,0)
weight_limit = 1000
sleigh_weight = 10

In [3]:
import pandas as pd
import numpy as np
from haversine import haversine
from operator import itemgetter

In [4]:
def weighted_trip_length(stops, weights): 
    tuples = [tuple(x) for x in stops.values]
    # adding the last trip back to north pole, with just the sleigh weight
    tuples.append(north_pole)
    weights.append(sleigh_weight)
    
    dist = 0.0
    prev_stop = north_pole
    prev_weight = sum(weights)
    for i in range(len(tuples)):        
        dist = dist + haversine(tuples[i], prev_stop) * prev_weight
        prev_stop = tuples[i]   
        prev_weight = prev_weight - weights[i]
    return dist

In [5]:
def weighted_reindeer_weariness(all_trips):
    uniq_trips = all_trips.TripId.unique()
    
    if any(all_trips.groupby('TripId').Weight.sum() > weight_limit):
        raise Exception("One of the sleighs over weight limit!")
 
    dist = 0
    for t in uniq_trips:
        this_trip = all_trips[all_trips.TripId==t]
        dist = dist + weighted_trip_length(this_trip[['Latitude','Longitude']], this_trip.Weight.tolist())
    
    return dist    

In [19]:
def get_worst_gift(trip):
    gifts = trip['GiftId'].values
    trip_wo_gifts = [trip.loc[trip['GiftId'] != gift] for gift in gifts]
    weight_wo_gifts = [weighted_reindeer_weariness(t) for t in trip_wo_gifts]
    return min(zip(gifts, weight_wo_gifts), key=itemgetter(1))[0]

In [7]:
def get_worst_gifts(all_trips):
    uniq_trips = all_trips.TripId.unique()
    return [get_worst_gift(all_trips[all_trips.TripId==t]) for t in uniq_trips]

In [8]:
gifts = pd.read_csv('./gifts.csv')
sample_sub = pd.read_csv('./sample_submission.csv')

In [9]:
all_trips = sample_sub.merge(gifts, on='GiftId')

In [16]:
def wrw_for_two_trips(trips, trip1, trip2):
    return weighted_reindeer_weariness(trips.loc[trips['TripId'].isin([trip1, trip2])])

In [ ]:
def get_trip(index):
    all_trips[all_trips.TripId==index]

In [85]:
uniq_trips = all_trips.TripId.unique()

In [13]:
from itertools import combinations as comb

In [84]:
print len(new_trips.TripId.unique())

5000


In [35]:
new_trips = all_trips.copy()
for i,j in comb(uniq_trips, 2):
    wrw_before = wrw_for_two_trips(new_trips, i, j)
    
    badtripi = new_trips.loc[new_trips['GiftId'] == get_worst_gift(new_trips.loc[new_trips['TripId'] == i])]
    badtripj = new_trips.loc[new_trips['GiftId'] == get_worst_gift(new_trips.loc[new_trips['TripId'] == j])]
    
    try:
        newer_trips = new_trips.copy()
        newer_trips.set_value(badtripi['TripId'], 'TripId', j)
        newer_trips.set_value(badtripj['TripId'], 'TripId', i)

        wrw_after = wrw_for_two_trips(newer_trips, i, j)
        #print i, j, get_worst_gift(new_trips.loc[new_trips['TripId']==i]), get_worst_gift(new_trips.loc[new_trips['TripId']==j]), wrw_before, wrw_after
        if wrw_after < wrw_before:
            print i, j, wrw_before, wrw_after
            new_trips = newer_trips
            
    except Exception:
        continue

0 1 58313522.4181 57305278.5953
0 61 65698891.0416 65184951.9664
0 83 72363285.4706 72193594.2079
0 88 68098191.6642 67080710.9312
0 89 85143431.4147 84001482.9745
0 102 68444256.1237 64917473.3855
0 103 77591783.2394 72843109.2584
0 107 68625763.4739 66184326.2821
0 125 80781037.6627 80324292.9495
0 157 80843897.3651 80575717.1688
0 166 81523972.8973 81214179.8213
0 184 94126938.3438 93419166.9455
0 202 96344294.7753 93321131.4476
0 208 87448940.0871 87028844.3081
0 708 84916526.3385 84681438.536
0 722 77541705.9921 77329135.5242
0 751 112806896.981 111298424.865
0 753 98748097.0037 98315559.5311
0 755 102606878.338 97470574.9618
0 762 95018420.5892 93287636.121
0 765 104478359.82 102842300.254
0 777 108146923.978 108143296.335
0 789 103402299.272 101031134.591
0 794 108450723.785 105680660.936
0 797 111307714.046 106820140.498
0 808 117180423.234 114123171.575
0 814 126170359.316 118942799.151
0 830 101923606.365 100532573.898
0 832 128209706.605 126107739.242
0 833 114793405.824 114

KeyboardInterrupt: 

In [105]:
import random
new_trips = all_trips.copy()
seeds = random.sample(uniq_trips, 5000)
k = 0

for i in seeds:
    print k
    k += 1
    bidders = random.sample(np.delete(uniq_trips, i), 20)
    for j in bidders:
        wrw_before = wrw_for_two_trips(new_trips, i, j)

        badtripi = new_trips.loc[new_trips['GiftId'] == get_worst_gift(new_trips.loc[new_trips['TripId'] == i])]
        badtripj = new_trips.loc[new_trips['GiftId'] == get_worst_gift(new_trips.loc[new_trips['TripId'] == j])]
        
        def move_bad_i_to_j(newer_trips):
            newer_trips.set_value(badtripi['TripId'], 'TripId', j)
        def move_bad_j_to_i(newer_trips):
            newer_trips.set_value(badtripj['TripId'], 'TripId', i)
        def swap_bad_gifts(newer_trips):
            move_bad_i_to_j(newer_trips)
            move_bad_j_to_i(newer_trips)
        
        def test_interchange(fn, new_trips):
            newer_trips = new_trips.copy()
            fn(newer_trips)
            try:
                wrw_after = wrw_for_two_trips(newer_trips, i, j)
            except:
                return False
            finally:
                if wrw_after < wrw_before:
                    #print i, j, wrw_before, wrw_after
                    new_trips = newer_trips
                    return new_trips
                else:
                    return False

        test1 = test_interchange(move_bad_i_to_j, new_trips) 
        if type(test1) != type(False):
            new_trips = test1
            continue
        test2 = test_interchange(move_bad_j_to_i, new_trips)
        if type(test2) != type(False):
            new_trips = test2
            continue
        test3 = test_interchange(swap_bad_gifts, new_trips)
        if type(test3) != type(False):
            new_trips = test3
            continue
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [37]:
weighted_reindeer_weariness(all_trips)

144525525772.40189

In [106]:
weighted_reindeer_weariness(new_trips)

139063126293.87006

In [115]:
new_trips.drop(['Longitude', 'Latitude', "Weight"], 1).set_index('GiftId').to_csv('sub3(1391e8).csv')

In [116]:
!ls

Untitled.ipynb        WRW_module.ipynb      sample_submission.csv
WRW.ipynb             WRW_module.py         sub1(1436e8).csv
WRW.py                WRW_module.pyc        sub2(1384e8).csv
WRW.pyc               gifts.csv             sub3(1391e8).csv


In [114]:
new_trips

,GiftId,TripId,Latitude,Longitude,Weight
0,1,1266,16.345769,6.303545,1.000000
1,2,264,12.494749,28.626396,15.524480
2,3,2,27.794615,60.032495,8.058499
3,4,3973,44.426992,110.114216,1.000000
4,5,4,-69.854088,87.946878,25.088892
5,6,4271,53.567970,-71.359308,38.000151
6,7,3324,12.902584,79.966949,44.206616
7,8,7,-6.291099,-64.891751,1.000000
8,9,2156,-2.685316,111.089758,1.000000
9,10,1128,38.428862,101.973671,35.701311
